<a href="https://colab.research.google.com/github/nastyaskutina/fairytales/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
filename = 'train.russian.v4_gold_conll'

In [ ]:
corpus = []
with open(filename) as f:
  corpus = [line.split() for line in f if line.strip() and not line.startswith("#")]

In [ ]:
pd.DataFrame()

""


In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(corpus)

In [ ]:
df[df[3] == 'забава!"'][6]

NameError: ignored

In [ ]:
' '.join(df[df[0] == '67'][3])

'В Мюнхене продолжается конференция по безопасности . 4 февраля на конференции выступил с речью известный американский сенатор - республиканец от штата Аризона Джон Маккейн . Ниже приводится часть речи Маккейна , которая касается отношений с Россией и развития восточной Европы . Некоторые отрывки из речи сенатора Маккейна Проблема с ядерной программой Ирана будет проверкой наших отношений с Россией . Я надеюсь , что Россия поддержит скорейшее решение Совета безопасности по этому вопросу . За пять месяцев до встречи « Большой восьмёрки » в Санкт - Петербурге , видно , что Москва желает быть признана великой державой . Эта возможность остаётся , так как есть много вещей , которые США и Европа могут делать совместно с Россией . Мы могли бы вместе противостоять угрозе Ирана , прекратить замороженные конфликты в восточной Европе , гарантировать стабильность и процветание Украины вместо противостояния в стиле « холодной войны » , способствовать преобразованиям в Средней Азии . Кремль , однак

In [ ]:
lst = list(df[df[0] == '1'][3])

In [ ]:
text = ' '.join(df[df[0] == '1'][3])
for item in text:
  if 

SyntaxError: ignored

In [ ]:
!pip install stanza
import stanza
stanza.download('ru')
nlp = stanza.Pipeline('ru')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 691 kB 5.1 MB/s 
     |████████████████████████████████| 216 kB 41.6 MB/s 
  Created wheel for emoji: filename=emoji-2.1.0-py3-none-any.whl size=212392 sha256=2f71a4f183ef54a96878f16eeef50b65e2abed5a0c64516379e2d9ba0a3eac5f
  Stored in directory: /root/.cache/pip/wheels/77/75/99/51c2a119f4cfd3af7b49cc57e4f737bed7e40b348a85d82804
Successfully built emoji


INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
nlp = stanza.Pipeline('ru')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [ ]:
doc = nlp(' '.join(df[df[0] == '1'][3]))

In [ ]:
def for_good_word(word, part_idx, ids, details, feats):
  if word.pos == 'VERB':
    ids.append('-')
    #all_verbs.append(word)
    #verb_ids.append(word.id)
    #events[word.lemma] = []
    details.append('VERB')
    feats.append(('-', '-', '-', '-'))
    
  elif word.deprel == 'obj' or word.deprel == 'nsubj':
    ids.append(part_idx)
    #all_participants.append(word)
    start_feats = ['-', '-', '-', '-']
    if word.feats:
      word_feats = word.feats.split('|')
      for feat in word_feats:
        if feat.split('=')[0] == 'Number':
          start_feats[0] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Gender':
          start_feats[1] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Person':
          start_feats[2] = feat.split('=')[1]
        elif feat.split('=')[0] == 'Animacy':
          start_feats[3] = feat.split('=')[1]
    feats.append(tuple(start_feats))
    if word.deprel == 'obj':
      details.append('obj')
    elif word.deprel == 'nsubj':
      details.append('nsubj')
      
  else:
    details.append('-')
    feats.append(('-', '-', '-', '-'))
    ids.append('-')
  return part_idx, ids, details, feats

In [ ]:
def add_inf1(doc, df1):
  #lst = list(df1[3])
  # event chain for sentence
  event_chain = []
  details = []
  feats = []
  #counter = 0
  part_idx = 0
  ids = []
  word_forms = {'text': [], 'lemmas': []}
  for sentence in doc.sentences:
    all_verbs = []
    verb_ids = []
    events = {}
    all_participants = []
    for word in sentence.words:
      part_idx += 1
      word_forms['text'].append(word.text)
      word_forms['lemmas'].append(word.lemma)
      part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)

      if word.pos == 'VERB':
        all_verbs.append(word)
        verb_ids.append(word.id)
        events[word.lemma] = []
      if word.deprel == 'obj' or word.deprel == 'nsubj':
        all_participants.append((word, part_idx))
    
    for participant, part_idx in all_participants:
      if participant.head in verb_ids:
        idx = verb_ids.index(participant.head)
        events[all_verbs[idx].lemma].append((participant.lemma, participant.deprel, part_idx))
    for key in events:
      if len(events[key]) != 0:
        event = (key, tuple(events[key]))
        event_chain.append(event)

  df1.loc[:, 'text'] = word_forms['text']
  df1.loc[:, 'lemma'] = word_forms['lemmas']
  df1.loc[:, 'details'] = details
  df1.loc[:, 'ids'] = ids
  df1.loc[:, 'Number'] = [f[0] for f in feats]
  df1.loc[:, 'Gender'] = [f[1] for f in feats]
  df1.loc[:, 'Person'] = [f[2] for f in feats]
  df1.loc[:, 'Animacy'] = [f[3] for f in feats]

  return df1, event_chain

In [ ]:
def add_inf(doc, df1):
  lst = list(df1[3])
  # event chain for sentence
  #event_chain = []
  details = []
  feats = []
  counter = 0
  part_idx = 0
  ids = []
  for sentence in doc.sentences:
    #all_verbs = []
    #verb_ids = []
    #events = {}
    #all_participants = []
    for word in sentence.words:
      part_idx += 1
      if word.text == lst[counter]:
        counter += 1
        part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)
      else:
        if (counter + 1) == len(lst):
          details.append('-')
          feats.append(('-', '-', '-', '-'))
          ids.append('-')
          break
        else:
          if word.text == lst[counter+1]:
            counter += 2
            details.append('-')
            feats.append(('-', '-', '-', '-'))
            ids.append('-')
            part_idx, ids, details, feats = for_good_word(word, part_idx, ids, details, feats)

  df1.loc[:, 'details'] = details
  df1.loc[:, 'ids'] = ids
  df1.loc[:, 'Number'] = [f[0] for f in feats]
  df1.loc[:, 'Gender'] = [f[1] for f in feats]
  df1.loc[:, 'Person'] = [f[2] for f in feats]
  df1.loc[:, 'Animacy'] = [f[3] for f in feats]
  return df1
    
"""    for participant in all_participants:
      if participant.head in verb_ids:
        idx = verb_ids.index(participant.head)
        events[all_verbs[idx].lemma].append((participant.lemma, participant.deprel))
    for key in events:
      if len(events[key]) != 0:
        event = (key, tuple(events[key]))
        event_chain.append(event)"""

'    for participant in all_participants:\n      if participant.head in verb_ids:\n        idx = verb_ids.index(participant.head)\n        events[all_verbs[idx].lemma].append((participant.lemma, participant.deprel))\n    for key in events:\n      if len(events[key]) != 0:\n        event = (key, tuple(events[key]))\n        event_chain.append(event)'

In [ ]:
df.loc[:, "E"] = list("abcd")

In [ ]:
df1 = df[df[0] == '1']
df1['details'] = details
df1['ids'] = ids
df1['Number'] = [f[0] for f in feats]
df1['Gender'] = [f[1] for f in feats]
df1['Person'] = [f[2] for f in feats]
df1['Animacy'] = [f[3] for f in feats]
df1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,0,1,2,3,4,5,6,7,8,9,10,11,details,ids,Number,Gender,Person,Animacy
0,1,0,0,Во,Sp-a,-,во,-,spk1,-,-,-,-,-,-,-,-,-
1,1,0,1,время,Ncnsan,-,время,-,spk1,-,-,-,-,-,-,-,-,-
2,1,0,2,своих,P---pga,-,свой,-,spk1,-,-,(1070),-,-,-,-,-,-
3,1,0,3,прогулок,Ncfpgn,-,прогулка,-,spk1,-,-,-,-,-,-,-,-,-
4,1,0,4,в,Sp-l,-,в,-,spk1,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,1,0,1,я,P-1-snn,-,я,-,spk1,-,-,(1070),nsubj,981,Sing,-,1,-
973,1,0,2,стал,Vmis-sma-p,-,стать,-,spk1,-,-,-,VERB,-,-,-,-,-
974,1,0,3,действовать,Vmn----a-e,-,действовать,-,spk1,-,-,-,VERB,-,-,-,-,-
975,1,0,4,решительно,R,-,решительно,-,spk1,-,-,-,-,-,-,-,-,-


In [ ]:
df1[:20]

,0,1,2,3,4,5,6,7,8,9,10,11,details,ids,Number,Gender,Person,Animacy
0,1,0,0,Во,Sp-a,-,во,-,spk1,-,-,-,-,-,-,-,-,-
1,1,0,1,время,Ncnsan,-,время,-,spk1,-,-,-,-,-,-,-,-,-
2,1,0,2,своих,P---pga,-,свой,-,spk1,-,-,(1070),-,-,-,-,-,-
3,1,0,3,прогулок,Ncfpgn,-,прогулка,-,spk1,-,-,-,-,-,-,-,-,-
4,1,0,4,в,Sp-l,-,в,-,spk1,-,-,-,-,-,-,-,-,-
5,1,0,5,окрестностях,Ncfpln,-,окрестность,-,spk1,-,-,-,-,-,-,-,-,-
6,1,0,6,Симеиза,Ncmsgn,-,симеиза,-,spk1,-,-,-,-,-,-,-,-,-
7,1,0,7,я,P-1-snn,-,я,-,spk1,-,-,(1070),nsubj,8,Sing,-,1,-
8,1,0,8,обратил,Vmis-sma-p,-,обратить,-,spk1,-,-,-,VERB,-,-,-,-,-
9,1,0,9,внимание,Ncnsan,-,внимание,-,spk1,-,-,-,obj,10,Sing,Neut,-,Inan


раздаем ид участникам + определяем их лицо число род и одушевленность если возможно
затем идем по айдишникам и попарно выписываем в новую таблицу лицо число род и одушевленность обоих участников + 1 если они кореферентны и 0 если нет
(+?ведем список всех попарных образований чтобы легче потом итерироваться было)

In [ ]:
df1[df1[11] != '-'][df1['Animacy'] != '-'].ids

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


60      61
155    156
219    220
286    287
314    315
371    372
375    376
423    424
491    492
567    570
587    590
653    660
655    662
681    688
686    693
699    706
715    722
717    724
722    729
725    732
737    744
743    750
776    783
833    840
863    870
Name: ids, dtype: object

In [ ]:
def make_coref_table(df1):
  big_dict = {'num1': [], 'num2': [], 'gen1': [], 'gen2': [], 
              'pers1': [],'pers2': [], 'anim1': [], 'anim2': [], 
              'coref': [],}
  list_id = list(df1[df1['ids'] != '-'].ids)
  for id1 in list_id:
    for id2 in list_id[list_id.index(id1)+1:]:
      big_dict['num1'].append(list(df1[df1['ids'] == id1].Number)[0])
      big_dict['num2'].append(list(df1[df1['ids'] == id2].Number)[0])
      big_dict['gen1'].append(list(df1[df1['ids'] == id1].Gender)[0])
      big_dict['gen2'].append(list(df1[df1['ids'] == id2].Gender)[0])
      big_dict['pers1'].append(list(df1[df1['ids'] == id1].Person)[0])
      big_dict['pers2'].append(list(df1[df1['ids'] == id2].Person)[0])
      big_dict['anim1'].append(list(df1[df1['ids'] == id1].Animacy)[0])
      big_dict['anim2'].append(list(df1[df1['ids'] == id2].Animacy)[0])
      ref1 = list(df1[11][df1['ids'] == id1])[0]
      ref2 = list(df1[11][df1['ids'] == id2])[0]
      for ref in [ref1, ref2]:
        if ref.startswith('('):
          ref = ref[1:]
        if ref.startswith(')', -1):
          ref = ref[:-1]
      coref = 0
      if ref1 == ref2:
        coref = 1
      big_dict['coref'].append(coref)
  return big_dict

In [ ]:
def make_coref_table1(df1):
  big_dict = {'id1': [], 'id2': [],
              'num1': [], 'num2': [], 'gen1': [], 'gen2': [], 
              'pers1': [],'pers2': [], 'anim1': [], 'anim2': []}
  list_id = list(df1[df1['ids'] != '-'].ids)
  for id1 in list_id:
    for id2 in list_id[list_id.index(id1)+1:]:
      big_dict['id1'].append(list(df1[df1['ids'] == id1].ids)[0])
      big_dict['id2'].append(list(df1[df1['ids'] == id2].ids)[0])
      big_dict['num1'].append(list(df1[df1['ids'] == id1].Number)[0])
      big_dict['num2'].append(list(df1[df1['ids'] == id2].Number)[0])
      big_dict['gen1'].append(list(df1[df1['ids'] == id1].Gender)[0])
      big_dict['gen2'].append(list(df1[df1['ids'] == id2].Gender)[0])
      big_dict['pers1'].append(list(df1[df1['ids'] == id1].Person)[0])
      big_dict['pers2'].append(list(df1[df1['ids'] == id2].Person)[0])
      big_dict['anim1'].append(list(df1[df1['ids'] == id1].Animacy)[0])
      big_dict['anim2'].append(list(df1[df1['ids'] == id2].Animacy)[0])
  return big_dict

In [ ]:
data = pd.DataFrame(big_dict)

In [ ]:
list(df1[11][df1['ids'] == 61])[0][:-1]

NameError: ignored

In [ ]:
data[data['coref'] == 1]

,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,coref
7,Sing,Sing,-,-,1,1,-,-,1
8,Sing,Sing,-,-,1,1,-,-,1
11,Sing,Sing,-,-,1,1,-,-,1
13,Sing,Sing,-,-,1,1,-,-,1
14,Sing,Sing,-,-,1,1,-,-,1
...,...,...,...,...,...,...,...,...,...
7122,Sing,Sing,-,Masc,1,3,-,-,1
7127,Sing,Sing,Masc,Masc,3,3,-,-,1
7130,Sing,Sing,Fem,Neut,-,-,Inan,Inan,1
7132,Sing,Sing,Fem,Fem,-,-,Inan,Inan,1


In [ ]:
df[df[0] == '67']

,0,1,2,3,4,5,6,7,8,9,10,11
35641,67,0,0,В,Sp-l,-,в,-,spk1,-,-,-
35642,67,0,1,Мюнхене,Ncmsln,-,мюнхен,-,spk1,-,-,-
35643,67,0,2,продолжается,Vmip3s-m-e,-,продолжаться,-,spk1,-,-,-
35644,67,0,3,конференция,Ncfsnn,-,конференция,-,spk1,-,-,(329
35645,67,0,4,по,Sp-d,-,по,-,spk1,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...
36314,67,0,11,север,Ncmsnn,-,север,-,spk1,-,-,-
36315,67,0,12,и,C,-,и,-,spk1,-,-,-
36316,67,0,13,центр,Ncmsnn,-,центр,-,spk1,-,-,-
36317,67,0,14,Европы,Ncfsgn,-,европа,-,spk1,-,-,(305)


In [ ]:
df2 = add_inf(nlp(' '.join(df[df[0] == '68'][3])), df[df[0] == '68'])
new_dic = make_coref_table(df2)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [ ]:
df2

,0,1,2,3,4,5,6,7,8,9,10,11,details,ids,Number,Gender,Person,Animacy
36319,68,0,0,В,Sp-a,-,в,-,spk1,-,-,-,-,-,-,-,-,-
36320,68,0,1,канун,Ncmsan,-,канун,-,spk1,-,-,-,-,-,-,-,-,-
36321,68,0,2,обнародования,Ncnsgn,-,обнародования,-,spk1,-,-,-,-,-,-,-,-,-
36322,68,0,3,президентом,Ncmsiy,-,президент,-,spk1,-,-,(304),-,-,-,-,-,-
36323,68,0,4,плана,Ncmsgn,-,план,-,spk1,-,-,(288,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36874,68,0,7,",",",",-,",",-,spk1,-,-,-,-,-,-,-,-,-
36875,68,0,8,так,P-----r,-,так,-,spk1,-,-,-,-,-,-,-,-,-
36876,68,0,9,в,Sp-a,-,в,-,spk1,-,-,-,-,-,-,-,-,-
36877,68,0,10,Афганистан,Ncmsan,-,афганистан,-,spk1,-,-,(303),-,-,-,-,-,-


In [ ]:
' '.join(df[df[0] == '11'][3])

'Что же это было ? Мальчик перестал ходить в школу , то есть он ходил , уезжал , ехал на метро , потом на маршрутном такси и дальше две остановки на троллейбусе ( трудный путь ) . А потом , оказывается , прогуливал . Возвращаться надо было тем же образом , три вида транспорта и десять минут пешком - - такова была цена перехода в новую хорошую школу . Туда пошел учиться обожаемый старший друг мальчика , еще с детского садика . И ребенок возроптал , что тоже пойдет туда же . Плакал всю весну . Родители похлопотали , особенно мать . Она написала письмо просвещенному директору этой школы , все по - честному . В прежней школе что - то не заладилось , начали придираться , классная руководительница звонила , новая математичка вообще стала сеять двойки , как пахарь в поле , широким жестом . И дорога была только в новую школу с гуманитарным уклоном . Мальчик , кроме того , ездил в музыкальную школу на метро плюс две остановки на автобусе и две на троллейбусе . Дважды в неделю . Вот там было все

In [ ]:
dic_list = []
for doc_id in list(df[0].unique()):
  try:
    df2 = add_inf(nlp(' '.join(df[df[0] == doc_id][3])), df[df[0] == doc_id])
    new_dic = make_coref_table(df2)
    dic_list.append(new_dic)
  except ValueError:
    continue

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [ ]:
len(dic_list)

171

In [ ]:
[1, 2][:1]

[1]

In [ ]:
pd.concat([pd.DataFrame(d) for d in dic_list], axis=0, join='outer', ignore_index=True)


,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,coref
0,Sing,Sing,-,Neut,1,-,-,Inan,0
1,Sing,Sing,-,Masc,1,-,-,Inan,0
2,Sing,Plur,-,Fem,1,-,-,Inan,0
3,Sing,Sing,-,Neut,1,-,-,Inan,0
4,Sing,Plur,-,Masc,1,-,-,Inan,0
...,...,...,...,...,...,...,...,...,...
7135,Sing,Sing,Neut,Fem,-,-,Inan,Inan,1
7136,Sing,Sing,Neut,-,-,1,Inan,-,0
7137,Sing,Sing,Masc,Fem,3,-,-,Inan,0
7138,Sing,Sing,Masc,-,3,1,-,-,0


In [ ]:
pd.concat([pd.DataFrame(d) for d in dic_list], axis=0, join='outer', ignore_index=True).to_csv('coref_inf.tsv', sep='\t') 

In [ ]:
coref_inf = pd.concat([pd.DataFrame(d) for d in dic_list], axis=0, join='outer', ignore_index=True)
coref_inf[coref_inf['coref'] == 1]

,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,coref
7,Sing,Sing,-,-,1,1,-,-,1
8,Sing,Sing,-,-,1,1,-,-,1
11,Sing,Sing,-,-,1,1,-,-,1
13,Sing,Sing,-,-,1,1,-,-,1
14,Sing,Sing,-,-,1,1,-,-,1
...,...,...,...,...,...,...,...,...,...
852667,-,Sing,-,Masc,-,-,-,Inan,1
852668,Sing,Sing,Masc,Masc,3,3,-,-,1
852675,Sing,Sing,Masc,Neut,-,-,Anim,Inan,1
852676,Sing,Sing,Masc,Masc,-,-,Anim,Inan,1


In [ ]:
import pandas as pd

In [ ]:
df_cat = pd.read_csv('coref_inf.tsv', sep='\t')  

In [ ]:
df_cat = df_cat.drop(['Unnamed: 0'], axis =1 )

In [ ]:
df_cat.anim1.unique()

array(['-', 'Inan', 'Anim'], dtype=object)

In [ ]:
feats = []
for item in df_cat.gen1:
  if item == '-':
    feats.append(0)
  elif item == 'Masc':
    feats.append(1)
  elif item == 'Fem':
    feats.append(2)
  elif item == 'Neut':
    feats.append(3)

df_cat.gen1 = feats

In [ ]:
feats = []
for item in df_cat.num1:
  if item == '-':
    feats.append(0)
  elif item == 'Sing':
    feats.append(1)
  elif item == 'Plur':
    feats.append(2)

df_cat.num1 = feats

In [ ]:
feats = []
for item in df_cat.pers1:
  if item == '-':
    feats.append(0)
  elif item == '1':
    feats.append(1)
  elif item == '2':
    feats.append(2)
  elif item == '3':
    feats.append(3)

df_cat.pers1 = feats

In [ ]:
feats = []
for item in df_cat.anim1:
  if item == '-':
    feats.append(0)
  elif item == 'Inan':
    feats.append(1)
  elif item == 'Anim':
    feats.append(2)

df_cat.anim1 = feats

In [ ]:
df_cat

,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,coref
0,1,1,0,3,1,0,0,1,0
1,1,1,0,1,1,0,0,1,0
2,1,2,0,2,1,0,0,1,0
3,1,1,0,3,1,0,0,1,0
4,1,2,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...
852673,1,1,1,3,3,0,0,1,0
852674,1,1,1,1,3,0,0,1,0
852675,1,1,1,3,0,0,2,1,1
852676,1,1,1,1,0,0,2,1,1


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split


columns=['coref']

X = df_cat.drop(columns, axis = 1)
y = df_cat['coref']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

knn = KNeighborsClassifier()


In [ ]:
from sklearn.linear_model import LogisticRegression


In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
gnb = LogisticRegression()
coref_model = gnb.fit(X_train, y_train)
y_pred = coref_model.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" 
      % (X_test.shape[0], (y_test != y_pred).sum()))
print((y_test != y_pred).sum() / X_test.shape[0] * 100)

Number of mislabeled points out of a total 85268 points : 22832
26.776750949946056


попытка применить к сказке

In [ ]:
text = '''Жил-был старик да старуха; детей у них не было. Уж чего они ни делали, как ни молились богу, а старуха все не рожала. Раз пошел старик в лес за грибами; попадается ему дорогою старый дед. «Я знаю, — говорит, — что у тебя на мыслях; ты все об детях думаешь. Поди-ка по деревне, собери с каждого двора по яичку и посади на те яйца клушку; что будет, сам увидишь!» Старик воротился в деревню; в ихней деревне был сорок один двор; вот он обошел все дворы, собрал с каждого по яичку и посадил клушку на сорок одно яйцо. Прошло две недели, смотрит старик, смотрит и старуха, — а из тех яичек народились мальчики; сорок крепких, здоровеньких, а один не удался — хил да слаб! Стал старик давать мальчикам имена; всем дал, а последнему не достало имени. «Ну, — говорит, — будь же ты Заморышек!»
Растут у старика со старухою детки, растут не по дням, а по часам; выросли и стали работа´ть, отцу с матерью помогать: сорок мо´лодцев в поле возятся, а Заморышек дома управляется. Пришло время сенокосное; братья траву косили, стога ставили, поработали с неделю и вернулись на деревню; поели, что бог послал, и легли спать. Старик смотрит и говорит: «Молодо-зелено! Едят много, спят крепко, а дела, поди, ничего не сделали!» — «А ты прежде посмотри, батюшка!» — отзывается Заморышек. Старик снарядился и поехал в луга; глянул — сорок стогов сметано: «Ай да молодцы ребята! Сколько за одну неделю накосили и в стога сметали».
На другой день старик опять собрался в луга, захотелось на свое добро полюбоваться; приехал — а одного стога как не бывало! Воротился домой и говорит: «Ах, детки! Ведь один стог-то пропал». — «Ничего, батюшка! — отвечает Заморышек.  — Мы этого вора поймаем; дай-ка мне сто рублев, а уж я дело сделаю». Взял у отца сто рублев и пошел к кузнецу: «Можешь ли сковать мне такую цепь, чтоб хватило с ног до головы обвить человека?» — «Отчего не сковать!» — «Смотри же, делай покрепче; коли цепь выдержит — сто рублев плачу, а коли лопнет — пропал твой труд!» Кузнец сковал железную цепь; Заморышек обвил ее вокруг себя, потянул — она и лопнула. Кузнец вдвое крепче сделал; ну, та годилась. Заморышек взял эту цепь, заплатил сто рублев и пошел сено караулить; сел под стог и дожидается.
Вот в самую полуночь поднялась погода, всколыхалось море, и выходит из морской глубины чудная кобылица, подбежала к первому стогу и принялась пожирать сено. Заморышек подскочил, обротал ее железной цепью и сел верхом. Стала его кобылица мыкать, по долам, по горам носить; нет, не в силах седока сбить! Остановилась она и говорит ему: «Ну, добрый мо´лодец, когда сумел ты усидеть на мне, то возьми-владей моими жеребятами». Подбежала кобылица к синю морю и громко заржала; тут сине море всколыхалося, и вышли на´ берег сорок один жеребец; конь коня лучше! Весь свет изойди, нигде таких не найдешь! Утром слышит старик на дворе ржанье, топот; что такое? а это его сынок Заморышек целый табун пригнал. «Здорово, — говорит, — братцы! Теперь у всех у нас по коню есть; поедемте невест себе искать». — «Поедем!» Отец с матерью благословили их, и поехали братья в путь-дорогу далекую.
Долго они ездили по белому свету, да где столько невест найти? Порознь жениться не хочется, чтоб никому обидно не было; а какая мать похвалится, что у ней как раз сорок одна дочь народилась? Заехали мо´лодцы за тридевять земель; смотрят: на крутой горе стоят белокаменные палаты, высокой стеной обведены, у ворот железные столбы поставлены. Сосчитали — сорок один столб. Вот они привязали к тем столбам своих богатырских коней и идут на двор. Встречает их баба-яга: «Ах вы, незваные-непрошеные! Как вы смели лошадей без спросу привязывать?» — «Ну, старая, чего кричишь? Ты прежде напой-накорми, в баню своди, да после про вести и спрашивай». Баба-яга накормила их, напоила, в баню сводила и стала спрашивать: «Что, добрые мо´лодцы, дела пытаете иль от дела лытаете?» — «Дела пытаем, бабушка!» — «Чего ж вам надобно?» — «Да невест ищем». — «У меня есть дочери», — говорит баба-яга, бросилась в высокие терема и вывела сорок одну де´вицу.
Тут они сосватались, начали пить, гулять, свадьбы справлять. Вечером пошел Заморышек на своего коня посмотреть. Увидел его добрый конь и промолвил человеческим голосом: «Смотри, хозяин! Как ляжете вы спать с молодыми женами, нарядите их в свои платья, а на себя наденьте женины; не то все пропадем!» Заморышек сказал это братьям; нарядили они молодых жен в свои платья, а сами оделись в женины и легли спать. Все заснули, только Заморышек глаз не смыкает.
В самую полночь закричала баба-яга зычным голосом: «Эй вы, слуги мои верные! Рубите незваным гостям буйны головы». Прибежали слуги верные и отрубили буйны головы дочерям бабы-яги. Заморышек разбудил своих братьев и рассказал все, что было; взяли они отрубленные головы, воткнули на железные спицы кругом стены´, потом оседлали коней и поехали наскоро.
Поутру встала баба-яга, глянула в окошечко — кругом стены´ торчат на спицах дочерние головы; страшно она озлобилась, приказала подать свой огненный щит, поскакала в погоню и начала палить щитом на все четыре стороны. Куда мо´лодцам спрятаться? Впереди сине море, позади баба-яга — и жжет и палит! Помирать бы всем, да Заморышек догадлив был: не забыл он захватить у бабы-яги платочек, махнул тем платочком перед собою — и вдруг перекинулся мост через все сине море; переехали добрые мо´лодцы на другую сторону. Заморышек махнул платочком в иную сторону — мост исчез, баба-яга воротилась назад, а братья домой поехали.'''

In [ ]:
text2 = '''Отслужил солдат три войны, не выслужил и выеденного яйца, и отпустили его вчистую. Вот он вышел на дорогу, шел, шел, пристал и сел у озера; сидит да думу думает: «Куда теперь мне деваться, чем прокормиться?.. К черту, что ли, в работники наняться!» Только вымолвил эти речи, а чертенок тут как тут — стоит перед ним, кланяется: «Здорово, служба!» — «Тебе что надо?» — «Да не сам ли ты захотел к нам в работники наняться? Что ж, служивый, наймись! Жалованье большое дадим».— «А какова работа?» — «Работа легкая: только пятнадцать лет не бриться, не стричься, соплей не сморкать, нос не утирать и одежи не переменять!» — «Ладно,— говорит солдат,— я возьмусь за эту работу, но с тем уговором, чтобы все мне было готово, чего душа пожелает!» — «Уж это как водится! Будь спокоен, за нами помешки не будет».— «Ну так по рукам! Сейчас же перенеси меня в большой столичный город да кучу денег притащи; ты ведь сам знаешь, что этого добра у солдата без малого ничего!»
Чертенок бросился в озеро, притащил кучу денег и мигом перенес солдата в большой город; перенес — и был таков! «Вот на дурака напал! — говорит солдат.— Еще не служил, не работал, а деньги взял». Нанял себе квартиру, не стрижется, не бреется, носа не утирает, одежи не переменяет, живет — богатеет; до того разбогател, что некуда стало денег девать. Что делать с серебром да с золотом? «Дай-ка,— вздумал он,— начну помогать бедным; пусть за мою душу молятся». Начал солдат раздавать деньги бедным, и направо дает, и налево дает — а денег у него не только не убывает, а еще прибавляется. Пошла об нем слава по всему царству, по всем людям.
Вот так-то жил солдат лет четырнадцать; на пятнадцатом году не хватило у царя казны; велел он позвать к себе этого солдата. Приходит к нему солдат небритый, немытый, нечесаный, сопли не  вытерты, одежа  не переменена.
«Здравия желаю, ваше величество!» — «Послушай, служивый! Ты, говорят, всем людям добро делаешь; дай мне хоть взаймы денег. У меня на жалованье войскам не хватает. Если дашь, сейчас тебя генералом пожалую».— «Нет, ваше величество, я генералом быть не желаю; а коли хочешь жаловать, отдай за меня одну из своих дочерей, и бери тогда казны, сколько надобно». Тут король призадумался; и дочерей жалко, и без денег обойтись нельзя. «Ну,— говорит,— хорошо; прикажи списать с себя портрет, я его дочерям покажу — которая за тебя пойдет?» Солдат повернулся, велел списать с себя портрет — точь-в-точь как он есть, и послал его к царю.
У того царя было три дочери, призвал их отец, показывает солдатский портрет старшей: «Пойдешь ли за него замуж? Он меня из великой нужды выведет». Царевна видит, что нарисовано страшилище, волоса всклокочены, ногти не выстрижены, сопли не вытерты! «Не хочу! — говорит.— Я лучше за черта пойду!» А черт откуда взялся — стоит позади с пером да с бумагой, услыхал это и записал ее душу. Спрашивает отец середнюю дочь: «Пойдешь за солдата замуж?» — «Как же! Я лучше в девках просижу, лучше с чертом повяжуся, чем за него идти!» Черт записал и другую душу. Спрашивает отец у меньшой дочери; она ему отвечает: «Видно, судьба моя такова! Иду за него замуж, а там что бог даст!»

Царь обрадовался, послал сказать солдату, чтоб к венцу готовился, и отправил к нему двенадцать подвод за золотом. Солдат потребовал к себе чертенка: «Вот двенадцать подвод — чтобы сейчас все были золотом насыпаны!» Чертенок побежал в озеро, и пошла у нечистых работа: кто мешок тащит, кто два; живой рукой насыпали воза и отправили к царю во дворец. Царь поправился и начал звать к себе солдата почитай каждый день, сажал с собою за единый стол, вместе с ним и пил и ел.
Вот, пока готовились они к свадьбе, прошло как раз пятнадцать лет: кончился срок солдатской службы. Зовет он чертенка и говорит: «Ну, служба моя покончилась: сделай теперь меня молодцом». Чертенок изрубил его на мелкие части, бросил в котел и давай варить; сварил, вынул и собрал все воедино как следует: косточка в косточку, суставчик в суставчик, жилка в жилку; потом взбрызнул мертвой и живой водою — и солдат встал таким молодцом, что ни в сказке сказать, ни пером написать. Обвенчался он с младшею царевною, и стали они жить-поживать, добра наживать; я на свадьбе был, мед-пиво пил, было у них вино — выпивал его по самое дно!
Прибежал чертенок в озеро; потребовал его дедушка к отчету: «Что, как солдат?» — «Отслужил свой срок верно и честно, ни разу не брился, не стригся, соплей не утирал, одежи не переменял». Рассердился на него дедушка: «В пятнадцать лет,— говорит,— не мог соблазнить ты солдата! Что даром денег потрачено, какой же ты черт после этого?» — и приказал бросить его в смолу кипучую. «Постой, дедушка! — отвечает внучек.— За солдатскую душу у меня две записаны».— «Как так?» — «Да вот как: задумал солдат на царевне жениться, так старшая да средняя сказали отцу, что лучше за черта пойдут замуж, чем за солдата! Стало быть, они — наши!» Дедушка оправил чертенка и велел его отпустить: знает-де свое дело!'''

In [ ]:
import pandas as pd

In [ ]:
table = pd.DataFrame()

In [ ]:
ft_table, ev_chain = add_inf1(nlp(f_texts[0]), table)
ft_dic = make_coref_table1(ft_table)

In [ ]:
ft = pd.DataFrame(ft_dic)

In [ ]:
feats = []
for item in ft.gen2:
  if item == '-':
    feats.append(0)
  elif item == 'Masc':
    feats.append(1)
  elif item == 'Fem':
    feats.append(2)
  elif item == 'Neut':
    feats.append(3)

ft.gen2 = feats

In [ ]:
feats = []
for item in ft.num2:
  if item == '-':
    feats.append(0)
  elif item == 'Sing':
    feats.append(1)
  elif item == 'Plur':
    feats.append(2)

ft.num2 = feats

In [ ]:
feats = []
for item in ft.pers2:
  if item == '-':
    feats.append(0)
  elif item == '1':
    feats.append(1)
  elif item == '2':
    feats.append(2)
  elif item == '3':
    feats.append(3)

ft.pers2 = feats

In [ ]:
feats = []
for item in ft.anim2:
  if item == '-':
    feats.append(0)
  elif item == 'Inan':
    feats.append(1)
  elif item == 'Anim':
    feats.append(2)

ft.anim2 = feats

In [ ]:
ft

,id1,id2,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2
0,6,11,1,1,2,2,0,0,2,0
1,6,12,1,0,2,0,0,0,2,0
2,6,18,1,1,2,2,0,3,2,0
3,6,20,1,1,2,2,0,0,2,2
4,6,29,1,1,2,3,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...
5351,522,527,2,1,3,2,0,0,1,2
5352,522,534,2,1,3,3,0,0,1,1
5353,524,527,1,1,2,2,0,0,2,2
5354,524,534,1,1,2,3,0,0,2,1


In [ ]:
columns=['id1', 'id2']
ft_X = ft.drop(columns, axis = 1)

In [ ]:
ft_y_pred = coref_model.predict(ft_X)

In [ ]:
ft['pred_coref'] = list(ft_y_pred)

In [ ]:
len(list(ft_y_pred))

5356

In [ ]:
ft[ft['pred_coref'] == 1]

,id1,id2,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,pred_coref
0,6,11,1,1,2,2,0,0,2,0,1
20,6,110,1,2,2,3,0,0,2,1,1
25,6,150,1,1,2,2,0,0,2,0,1
77,6,391,1,2,2,3,0,0,2,1,1
99,6,522,1,2,2,3,0,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
5346,501,522,2,2,2,3,0,0,1,1,1
5349,501,534,2,1,2,3,0,0,1,1,1
5350,522,524,2,1,3,2,0,0,1,2,1
5351,522,527,2,1,3,2,0,0,1,2,1


In [ ]:
ft_table[ft_table['ids'] == 398]

,text,lemma,details,ids,Number,Gender,Person,Animacy
537,высокий,высокий,nsubj,398,Sing,Masc,-,-


In [ ]:
ev_chain

[('делать', (('родной', 'nsubj', 11), ('что', 'nsubj', 12))),
 ('гладеть', (('она', 'obj', 18),)),
 ('угождать', (('падчерица', 'nsubj', 20),)),
 ('сказать', (('правда', 'obj', 38),)),
 ('купаться', (('она', 'nsubj', 48),)),
 ('умываться', (('день', 'nsubj', 60),)),
 ('делать', (('что', 'obj', 61),)),
 ('пошумить', (('ветер', 'nsubj', 62),)),
 ('расходиться', (('баба', 'nsubj', 70),)),
 ('придумывать', (('все', 'nsubj', 77),)),
 ('чесать', (('зуб', 'obj', 81),)),
 ('придумать', (('мачеха', 'obj', 84), ('падчерица', 'obj', 85))),
 ('хотеть', (('она', 'obj', 97),)),
 ('видать', (('глаз', 'nsubj', 103), ('она', 'obj', 104))),
 ('слыхать', (('ухо', 'nsubj', 110),)),
 ('затужить', (('старик', 'nsubj', 111),)),
 ('сажать', (('дочка', 'obj', 118),)),
 ('повезти', (('поле', 'obj', 134),)),
 ('видать', (('глаз', 'nsubj', 148),)),
 ('остаться', (('бедненький', 'nsubj', 150),)),
 ('творить', (('молитва', 'obj', 155),)),
 ('приходить', (('Мороз', 'nsubj', 157),)),
 ('принести', (('бог', 'nsubj', 1

In [ ]:
for ev in ev_chain:
  if ev[0] == 'быть' or ev[0] == 'пытать':
    print(ev)

('быть', (('ребенок', 'nsubj', 6),))
('быть', (('что', 'nsubj', 66),))
('быть', (('двор', 'nsubj', 78),))
('пытать', (('что', 'nsubj', 616), ('дело', 'obj', 621)))
('пытать', (('дело', 'nsubj', 624),))
('быть', (('дочь', 'nsubj', 642),))
('быть', (('что', 'nsubj', 748),))


In [ ]:
ft_cor = {}
for i, row in ft[['id1', 'id2']][ft['pred_coref'] == 1].iterrows():
  if row['id1'] not in ft_cor:
    ft_cor[row['id1']] = []
  ft_cor[row['id1']].append(row['id2'])

  if row['id2'] not in ft_cor:
    ft_cor[row['id2']] = []
  ft_cor[row['id2']].append(row['id1']) 

In [ ]:
ft_cor = {}
for i, row in ft[['id1', 'id2']][ft['pred_coref'] == 1].iterrows():
  if row['id1'] not in ft_cor:
    ft_cor[row['id1']] = []
  ft_cor[row['id1']].append(row['id2'])

In [ ]:
что если создать словарь и зааппендить в него все корефы этого

3

In [ ]:
ft_cor[398]

In [ ]:
trio = {}

In [ ]:
for item1 in ft_cor:
  for item2 in ft_cor[item1]:
    tyk = ft_cor[item1] + ft_cor[item2]
    trio[item1] = list(set(tyk))

In [ ]:
len(trio[8])

4

In [ ]:
sets = []

In [ ]:
for item in trio:
  nset = trio[item] + [item]
  nset = list(set(nset))
  nset.sort()
  if tuple(nset) not in sets:
    sets.append(tuple(nset))

In [ ]:
to_del = []
for k, s in enumerate(sets):
  if k + 1 != len(sets):
    if set(sets[k+1]) & set(s) == set(sets[k+1]):
      if set(sets[k+1]) not in to_del:
        to_del.append(set(sets[k+1]))
    elif set(sets[k+1]) & set(s) == set(s):
      if set(s) not in to_del:
        to_del.append(set(s))

In [ ]:
tuple(to_del[0])

(891, 811, 668, 1325)

In [ ]:
for el in to_del:
  sets.remove(tuple(el))

ValueError: ignored

In [ ]:
sets

[(8, 668, 811, 891, 1325),
 (668, 811, 891, 1325),
 (17,
  33,
  44,
  97,
  113,
  133,
  149,
  211,
  282,
  405,
  417,
  468,
  470,
  476,
  478,
  506,
  509,
  530,
  550,
  558,
  572,
  576,
  600,
  615,
  628,
  654,
  668,
  721,
  755,
  776,
  806,
  811,
  820,
  833,
  844,
  856,
  891,
  915,
  925,
  936,
  952,
  963,
  967,
  993,
  1018,
  1025,
  1026,
  1043,
  1049,
  1051,
  1084,
  1091,
  1125,
  1141,
  1145,
  1198,
  1214,
  1260,
  1276,
  1306,
  1311,
  1325,
  1357,
  1372,
  1423,
  1433,
  1445,
  1533,
  1665,
  1690,
  1700),
 (44,
  97,
  113,
  133,
  211,
  282,
  468,
  470,
  476,
  530,
  572,
  600,
  615,
  668,
  755,
  806,
  811,
  891,
  936,
  952,
  967,
  1043,
  1051,
  1084,
  1145,
  1198,
  1214,
  1260,
  1276,
  1306,
  1325,
  1433,
  1533,
  1665,
  1690),
 (33,
  44,
  97,
  113,
  133,
  211,
  282,
  468,
  470,
  476,
  530,
  572,
  600,
  615,
  668,
  755,
  806,
  811,
  891,
  936,
  952,
  967,
  1043,
  1051,
  1

In [ ]:
narrative_chains = {}
for event in ev_chain:
  for part in event[1]:
    if part[0] not in narrative_chains:
      narrative_chains[part[0]] = [(event[0], part[1])]
    else:
      narrative_chains[part[0]].append((event[0], part[1]))

{1, 2, 4}

In [ ]:
rep_list = []
for ft in ft_cor:
  if ft_cor[ft] in rep_list:
    print(ft, ft_cor[ft])
  else:
    rep_list.append(ft_cor[ft])

19 [129, 199, 359, 587, 590]
124 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
244 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
338 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
444 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
467 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
546 [48, 115, 129, 169, 199, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
555 [48, 85, 115, 129, 169, 176, 199, 277, 282, 359, 370, 379, 396, 431, 448, 462, 541, 587, 590, 629, 658, 659, 662, 694, 701, 708, 716, 738, 744]
597 [48, 85, 115, 1

[('быть', (('ребенок', 'nsubj', 6),)),
 ('делать', (('они', 'nsubj', 9),)),
 ('рожать', (('старуха', 'nsubj', 19),)),
 ('пойти', (('старик', 'nsubj', 22),)),
 ('попадаться', (('дед', 'nsubj', 32),)),
 ('знать', (('я', 'nsubj', 34),)),
 ('думать', (('ты', 'nsubj', 47), ('все', 'obj', 48))),
 ('посадить', (('клушка', 'obj', 65),)),
 ('быть', (('что', 'nsubj', 67),)),
 ('воротиться', (('старик', 'nsubj', 68),)),
 ('обойти', (('он', 'nsubj', 82), ('двор', 'obj', 85))),
 ('сажать', (('клушка', 'obj', 94),)),
 ('пройти', (('неделя', 'nsubj', 97),)),
 ('смотреть', (('старик', 'nsubj', 100), ('старуха', 'nsubj', 104))),
 ('народиться', (('мальчик', 'nsubj', 112),)),
 ('удаться', (('крепкий', 'nsubj', 115), ('один', 'nsubj', 120))),
 ('стать', (('старик', 'nsubj', 126),)),
 ('давать', (('имя', 'obj', 129),)),
 ('расти', (('детка', 'nsubj', 145),)),
 ('возиться', (('мо´лодцев', 'nsubj', 167),)),
 ('прийти', (('время', 'nsubj', 169),)),
 ('косить', (('брат', 'nsubj', 172), ('трава', 'obj', 173)))

In [ ]:
def coref_check_a(id1, id2, antecendent, ft_table, corefs):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    word_feat3 = list(ft_table[ft_table['ids'] == antecendent][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
    if word_feat1 != word_feat3 and word_feat1 != '-' and word_feat3 != '-':
      counter += 1
  if counter < 1:
    corefs[id1] = antecendent


def coref_check(id1, id2, ft_table, corefs):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
  if counter < 1:
    corefs[id1] = id2

In [ ]:
# другой вариант но скорее всего тоже нерабочий
narrative_chains = {}
all_events = []
corefs = {}
for event in ev_chain:
  for part in event[1]:
    all_events.append((event[0], part[1]))
    if part[2] in ft_cor:
      if part[2] in corefs:
        antecendent = corefs[part[2]]
        antecendent_name = list(ft_table[ft_table['ids'] == antecendent].lemma)[0]
        narrative_chains[antecendent_name]['names'].append(part[0])
        narrative_chains[antecendent_name]['events'].append((event[0], part[1]))
        for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
          coref_check_a(idx, part[2], antecendent, ft_table, corefs)
      else:
        for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
          coref_check(idx, part[2], ft_table, corefs)
    
    if part[0] in narrative_chains:
      narrative_chains[part[0]]['events'].append((event[0], part[1]))
    else:
      narrative_chains[part[0]] = {}
      narrative_chains[part[0]]['names'] = []
      narrative_chains[part[0]]['events'] = [(event[0], part[1])]

In [ ]:
0 < 1

True

In [ ]:
def coref_check_n(id1, id2, ft_table):
  counter = 0
  for feat in ['Number', 'Gender', 'Animacy']:
    word_feat1 = list(ft_table[ft_table['ids'] == id1][feat])[0]
    word_feat2 = list(ft_table[ft_table['ids'] == id2][feat])[0]
    if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
      counter += 1
  result = counter < 1
  return result

In [ ]:
def coref_change(id1, id2, corefs):
  for k in corefs:
    if id1 == corefs[k]:
      corefs[k] = id2

In [ ]:
# другой вариант но скорее всего тоже нерабочий (версия с другой структурой данных)
narrative_chains = {}
all_events = []
corefs = {}
for event in ev_chain:
  for part in event[1]:
    all_events.append((event[0], part[1]))
    chain_counter = 0
    if part[2] in ft_cor:
      if part[2] in corefs:
        if part[2] == 397:
          print('1')
        antecendent = corefs[part[2]]
        #antecendent_name = list(ft_table[ft_table['ids'] == antecendent].lemma)[0]
        narrative_chains[antecendent]['names'].append(part[0])
        narrative_chains[antecendent]['events'].append((event[0], part[1], part[0]))
        chain_counter = 1
        for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
          coref_check_a(idx, part[2], antecendent, ft_table, corefs)
      else:
        if part[2] == 397:
          print('2')
        for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
          coref_check(idx, part[2], ft_table, corefs)
    
# не дает покоя что тут фигня какая то подумай над тем как работает присутствие в нарратив чейнах на то что проходит условие сврху   
    '''else:
      for key in narrative_chains:
        if part[0] in narrative_chains[key]['names']:
          # проверка
          if coref_check_n(key, part[2], ft_table):
            narrative_chains[key]['names'].append(part[0])
            narrative_chains[key]['events'].append((event[0], part[1], part[0]))
            chain_counter = 1
            break
          else:
            continue'''
    
    if chain_counter == 0:
      for key in narrative_chains:
        if part[0] in narrative_chains[key]['names']:
            # проверка
          if coref_check_n(key, part[2], ft_table):
            narrative_chains[key]['names'].append(part[0])
            narrative_chains[key]['events'].append((event[0], part[1], part[0]))
            coref_change(part[2], key, corefs)
            chain_counter = 1
            break
          else:
            continue
      
    if chain_counter == 0:
      narrative_chains[part[2]] = {}
      narrative_chains[part[2]]['names'] = [part[0]]
      narrative_chains[part[2]]['events'] = [(event[0], part[1], part[0])]

2


In [ ]:
corefs[397]

KeyError: ignored

In [ ]:
# один вариант но оч плохой
narrative_chains = {}
corefs = {}
for event in ev_chain:
  for part in event[1]:
    part_dic = {'other_names': []}
    if part[2] in ft_cor:
      if part[2] not in corefs:
        for idx in ft_cor[part[2]]:
          if list(ft_table[ft_table['ids'] == idx].lemma)[0] != list(ft_table[ft_table['ids'] == part[2]].lemma)[0]:
            counter = 0
            for feat in ['Number', 'Gender', 'Animacy']:
              word_feat1 = list(ft_table[ft_table['ids'] == idx][feat])[0]
              word_feat2 = list(ft_table[ft_table['ids'] == part[2]][feat])[0]
              if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
                counter += -1
            if counter > -1:
              part_dic['other_names'].append(list(ft_table[ft_table['ids'] == idx].lemma)[0])
              if idx > part[2]:
                corefs[idx] = part[2]
        if part[0] not in narrative_chains:
          part_dic['events'] = [(event[0], part[1])]
          narrative_chains[part[0]] = part_dic
        else:
          narrative_chains[part[0]]['events'].append((event[0], part[1]))
      else:
        antecedent = list(ft_table[ft_table['ids'] == corefs[part[2]]].lemma)[0]
        narrative_chains[antecedent]['events'].append((event[0], part[1]))

    else:
      if part[0] not in narrative_chains:
        part_dic['events'] = [(event[0], part[1])]
        narrative_chains[part[0]] = part_dic
      else:
        narrative_chains[part[0]]['events'].append((event[0], part[1]))

In [ ]:
all_events[:10]

[('жил-бнуть', 'nsubj'),
 ('быть', 'nsubj'),
 ('делать', 'nsubj'),
 ('рожать', 'nsubj'),
 ('пойти', 'nsubj'),
 ('попадаться', 'nsubj'),
 ('знать', 'nsubj'),
 ('думать', 'nsubj'),
 ('поднить', 'obj'),
 ('посадить', 'obj')]

In [ ]:
narrative_chains

{2: {'names': ['старик',
   'старик',
   'старик',
   'старик',
   'старик',
   'старик',
   'старик',
   'старик',
   'старик'],
  'events': [('жил-бнуть', 'nsubj', 'старик'),
   ('пойти', 'nsubj', 'старик'),
   ('воротиться', 'nsubj', 'старик'),
   ('смотреть', 'nsubj', 'старик'),
   ('стать', 'nsubj', 'старик'),
   ('смотреть', 'nsubj', 'старик'),
   ('снарядиться', 'nsubj', 'старик'),
   ('собраться', 'nsubj', 'старик'),
   ('слышать', 'nsubj', 'старик')]},
 6: {'names': ['ребенок'], 'events': [('быть', 'nsubj', 'ребенок')]},
 9: {'names': ['они',
   'они',
   'они',
   'они',
   'они',
   'они',
   'они',
   'они',
   'они',
   'они'],
  'events': [('делать', 'nsubj', 'они'),
   ('благословить', 'obj', 'они'),
   ('ездить', 'nsubj', 'они'),
   ('привязать', 'nsubj', 'они'),
   ('встречать', 'obj', 'они'),
   ('накормить', 'obj', 'они'),
   ('сосваться', 'nsubj', 'они'),
   ('нарядить', 'obj', 'они'),
   ('нарядить', 'nsubj', 'они'),
   ('брать', 'nsubj', 'они')]},
 19: {'names': [

In [ ]:
coref_event_count = []
coref_role_count = []
for cor in narrative_chains:
  events = narrative_chains[cor]['events']
  names = narrative_chains[cor]['names']
  name = None
  for item in Counter(names).most_common():
    for p in morph.parse(item[0]):
      if 'NOUN' in p.tag:
        name = item[0]
        break
    if name != None:
      break
  
  if len(events) > 1:
    for ev1 in events[:-1]:
      for ev2 in events[events.index(ev1)+1:]:
        coref_event_count.append((ev1[:-1], ev2[:-1]))
        if name != None:
          coref_role_count.append((ev1[:-1], ev2[:-1], name))


580

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 8.2 MB 11.3 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=b610671b3a2c4563d5fa2ad3c848b17189683dac1fa6028e78eb55ebb719700f
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [ ]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
morph.parse('мама')


[Parse(word='мама', tag=OpencorporaTag('NOUN,anim,femn sing,nomn'), normal_form='мама', score=1.0, methods_stack=((DictionaryAnalyzer(), 'мама', 1988, 0),))]

In [ ]:
'NOUN' in morph.parse('мама')[0].tag

True

In [ ]:
from collections import Counter

In [ ]:
Counter(coref_event_count).most_common(5)

In [ ]:
len(Counter(coref_event_count).most_common())

512

In [ ]:
ft_table[ft_table['ids'] == 827]

,text,lemma,details,ids,Number,Gender,Person,Animacy
1152,баба,баба,nsubj,827,Sing,Fem,-,Anim


In [ ]:
some_dic = {'other_names': []}
for idx in ft_cor[46]:
  if list(ft_table[ft_table['ids'] == idx].lemma)[0] != list(ft_table[ft_table['ids'] == 46].lemma)[0]:
    counter = 0
    for feat in ['Number', 'Gender', 'Animacy']:
      word_feat1 = list(ft_table[ft_table['ids'] == idx][feat])[0]
      word_feat2 = list(ft_table[ft_table['ids'] == 46][feat])[0]
      if word_feat1 != word_feat2 and word_feat1 != '-' and word_feat2 != '-':
        counter += -1
    if counter > -1:
      some_dic['other_names'].append(list(ft_table[ft_table['ids'] == idx].lemma)[0])

some_dic   

{'other_names': ['слепый', 'этот']}

In [ ]:
a = [1, 3, 4]
for b in [c for c in a if c > 2]:
  print(b)

3
4


In [ ]:
ft_table[ft_table['ids'] == 745]

,text,lemma,details,ids,Number,Gender,Person,Animacy
1045,баба,баба,nsubj,745,Sing,Fem,-,Anim


pmi???????



In [ ]:
pair = coref_event_count[0]

In [ ]:
Counter(all_events)[('пытать', 'nsubj')]

2

In [ ]:
math.log((4 / 580) / ((5 / 144)  * (2 / 144)), 2)

3.838012817982328

In [ ]:
math.log(0.006896551724137931 / (0.034722222222222224  * 0.013888888888888888), 2)

3.838012817982328

In [ ]:
len(all_events)

121

In [ ]:
len(ev_chain)

107

In [ ]:
len(coref_event_count)

476

In [ ]:
import math
from tqdm import tqdm

In [ ]:

for i in tqdm(range(10000)):

In [ ]:
pmi((('быть', 'nsubj'), ('пытать', 'nsubj')), coref_event_count, all_events)

0.034722222222222224 0.013888888888888888 0.006896551724137931


3.838012817982328

In [ ]:
def pmi(pair, c_coref_event_count, c_all_events, event_counter, len_coref_event):
  p1 = c_all_events[pair[0]] / event_counter
  p2 = c_all_events[pair[1]] / event_counter
  p_pair = c_coref_event_count[pair] / len_coref_event
  if (p1 * p2) == 0:
    result = 0
  elif p_pair / (p1 * p2) <= 0:
    result = 0
  else:
    result = math.log(p_pair / (p1 * p2), 2)
  return result

In [ ]:
with open('event_count.txt', 'r', encoding='utf-8') as f:
  counts = f.read().split()

event_counter = 0
for c in counts:
  event_counter += int(c)

In [ ]:
for t in Counter([1, 2, 3, 4]):
  print(t)

1
2
3
4


In [ ]:
len(Counter(coref_event_count))

185379

In [ ]:
with open('coref_event_count.txt', 'r', encoding='utf-8') as f:
  cor_lines = f.readlines()

coref_event_count = []
for line in cor_lines:
  coref_event_count.append((tuple(line.split()[:2]), tuple(line.split()[-2:])))

In [ ]:
with open('coref_role_count.txt', 'r', encoding='utf-8') as f:
  cor_lines = f.readlines()

coref_role_count = []
for line in cor_lines:
  coref_role_count.append((tuple(line.split()[:2]), tuple(line.split()[-3:-1]), line.split()[-1]))

In [ ]:
coref_role_count[:5]

[(('делать', 'nsubj'), ('делать', 'nsubj'), 'дочь'),
 (('делать', 'nsubj'), ('угождать', 'nsubj'), 'дочь'),
 (('делать', 'nsubj'), ('сказать', 'obj'), 'дочь'),
 (('делать', 'nsubj'), ('делать', 'obj'), 'дочь'),
 (('делать', 'nsubj'), ('расходиться', 'nsubj'), 'дочь')]

In [ ]:
with open('all_events.txt', 'r', encoding='utf-8') as f:
  ev_lines = f.readlines()

all_events = []
for line in ev_lines:
  all_events.append(tuple(line.split()))

In [ ]:
pmi_scores = {}
c_coref_event_count = Counter(coref_event_count)
c_all_events = Counter(all_events)
len_coref_event = len(coref_event_count)
for pair in tqdm(c_coref_event_count):
  pmi_scores[pair] = pmi(pair, c_coref_event_count, c_all_events, event_counter, len_coref_event)

100%|██████████| 185379/185379 [00:00<00:00, 408113.54it/s]


In [ ]:
nar_chain = [('брать', 'obj'), ('испугаться', 'obj'), ('сослужить', 'obj')]
pmi_sc = 0
new_ev = None
for ev in c_all_events:
  pmi_list = []
  for chain_ev in nar_chain:
    pmi_list.append(pmi((chain_ev, ev), c_coref_event_count, c_all_events, event_counter, len_coref_event))
  if sum(pmi_list) > pmi_sc:
    pmi_sc = sum(pmi_list)
    new_ev = ev

print(new_ev)

('сослужить', 'obj')


In [ ]:
c_all_events.most_common(5)

[(('говорить', 'nsubj'), 928),
 (('брать', 'obj'), 848),
 (('стать', 'nsubj'), 528),
 (('быть', 'nsubj'), 491),
 (('брать', 'nsubj'), 476)]

In [ ]:
max_pmi = 0
for key in pmi_scores:
  max_pmi = max(max_pmi, pmi_scores[key])


In [ ]:
for key in pmi_scores:
  if pmi_scores[key] == max_pmi:
    print(key)

In [ ]:
max_pmi

9.233160649024448

In [ ]:
pmi_scores[(('быть', 'nsubj'), ('пытать', 'nsubj'))]

3.838012817982328

In [ ]:
all_events[0]

('делать', 'nsubj')

several texts

In [ ]:
def for_gen(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Masc':
      feats.append(1)
    elif item == 'Fem':
      feats.append(2)
    elif item == 'Neut':
      feats.append(3)
      
  if c[0] == 'gen1':
    ft.gen1 = feats
  elif c[0] == 'gen2':
    ft.gen2 = feats

In [ ]:
def for_num(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Sing':
      feats.append(1)
    elif item == 'Plur':
      feats.append(2)
      
  if c[0] == 'num1':
    ft.num1 = feats
  elif c[0] == 'num2':
    ft.num2 = feats

In [ ]:
def for_pers(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == '1':
      feats.append(1)
    elif item == '2':
      feats.append(2)
    elif item == '3':
      feats.append(3)
  
  if c[0] == 'pers1':
    ft.pers1 = feats
  elif c[0] == 'pers2':
    ft.pers2 = feats

In [ ]:
def for_anim(c, ft):
  feats = []
  for item in c[1]:
    if item == '-':
      feats.append(0)
    elif item == 'Inan':
      feats.append(1)
    elif item == 'Anim':
      feats.append(2)
  
  if c[0] == 'anim1':
    ft.anim1 = feats
  elif c[0] == 'anim2':
    ft.anim2 = feats

In [ ]:
def table_fix(ft):
  cols = [('gen1', ft.gen1), ('gen2', ft.gen2), ('num1', ft.num1), ('num2', ft.num2), 
          ('pers1', ft.pers1), ('pers2', ft.pers2), ('anim1', ft.anim1), ('anim2', ft.anim2)]
  for c in cols:
    if c[0].startswith('gen'):
      for_gen(c, ft)
    elif c[0].startswith('num'):
      for_num(c, ft)
    elif c[0].startswith('pers'):
      for_pers(c, ft)
    elif c[0].startswith('anim'):
      for_anim(c, ft)


In [ ]:
def text_preproc(text):
  table = pd.DataFrame()
  ft_table, ev_chain = add_inf1(nlp(text), table)
  ft_dic = make_coref_table1(ft_table)
  ft = pd.DataFrame(ft_dic)
  table_fix(ft)
  columns=['id1', 'id2']
  ft_X = ft.drop(columns, axis = 1)
  ft_y_pred = coref_model.predict(ft_X)
  ft['pred_coref'] = list(ft_y_pred)
  return ft, ev_chain, ft_table

In [ ]:
def get_coref_chains(ft, ev_chain, ft_table):
  ft_cor = {}
  for i, row in ft[['id1', 'id2']][ft['pred_coref'] == 1].iterrows():
    if row['id1'] not in ft_cor:
      ft_cor[row['id1']] = []
    ft_cor[row['id1']].append(row['id2'])

    if row['id2'] not in ft_cor:
      ft_cor[row['id2']] = []
    ft_cor[row['id2']].append(row['id1']) 
  

  coref_chains = {}
  all_events = []
  corefs = {}
  for event in ev_chain:
    for part in event[1]:
      all_events.append((event[0], part[1]))
      chain_counter = 0
      if part[2] in ft_cor:
        if part[2] in corefs:
          antecendent = corefs[part[2]]
          coref_chains[antecendent]['names'].append(part[0])
          coref_chains[antecendent]['events'].append((event[0], part[1], part[0]))
          chain_counter = 1
          for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
            coref_check_a(idx, part[2], antecendent, ft_table, corefs)
        else:
          for idx in [idx for idx in ft_cor[part[2]] if idx > part[2]]:
            coref_check(idx, part[2], ft_table, corefs)
      
      if chain_counter == 0:
        for key in coref_chains:
          if part[0] in coref_chains[key]['names']:
              # проверка
            if coref_check_n(key, part[2], ft_table):
              coref_chains[key]['names'].append(part[0])
              coref_chains[key]['events'].append((event[0], part[1], part[0]))
              coref_change(part[2], key, corefs)
              chain_counter = 1
              break
            else:
              continue
        
      if chain_counter == 0:
        coref_chains[part[2]] = {}
        coref_chains[part[2]]['names'] = [part[0]]
        coref_chains[part[2]]['events'] = [(event[0], part[1], part[0])]
  
  return coref_chains, all_events

In [ ]:
text_preproc(text)

11781


In [ ]:
def add_data(coref_chains, coref_event_count, coref_role_count):
  for cor in coref_chains:
    events = coref_chains[cor]['events']
    names = coref_chains[cor]['names']
    name = None
    for item in Counter(names).most_common():
      for p in morph.parse(item[0]):
        if 'NOUN' in p.tag:
          name = item[0]
          break
      if name != None:
        break
    
    if len(events) > 1:
      for ev1 in events[:-1]:
        for ev2 in events[events.index(ev1)+1:]:
          coref_event_count.append((ev1[:-1], ev2[:-1]))
          if name != None:
            coref_role_count.append((ev1[:-1], ev2[:-1], name))

In [ ]:
coref_event_count = []
coref_role_count = []
all_events = []
event_counter = 0
for t in [text, text2]:
  ft, ev_chain, ft_table = text_preproc(t)
  coref_chains, text_events = get_coref_chains(ft, ev_chain, ft_table)
  event_counter += len(ev_chain)
  all_events.extend(text_events)
  add_data(coref_chains, coref_event_count, coref_role_count)
  print('ready')

ready
ready


In [ ]:
coref_chains, text_events = get_coref_chains(ft, ev_chain)

In [ ]:
ft

,id1,id2,num1,num2,gen1,gen2,pers1,pers2,anim1,anim2,pred_coref
0,2,10,1,1,1,3,0,0,2,1,0
1,2,14,1,1,1,1,0,3,2,0,0
2,2,16,1,1,1,1,0,3,2,0,0
3,2,33,1,1,1,2,0,0,2,1,0
4,2,48,1,2,1,2,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9175,751,756,1,1,1,1,0,3,2,0,0
9176,751,763,1,1,1,3,0,0,2,1,0
9177,753,756,1,1,1,1,0,3,1,0,0
9178,753,763,1,1,1,3,0,0,1,1,1


In [ ]:
event_counter

206

In [ ]:
coref_role_count[0]

(('делать', 'nsubj'), ('делать', 'nsubj'), 'дочь')

In [ ]:
len(all_events)

239

In [ ]:
Counter(coref_role_count).most_common(10)

[((('говорить', 'nsubj'), ('говорить', 'nsubj'), 'образ'), 55),
 ((('говорить', 'nsubj'), ('говорить', 'nsubj'), 'буря'), 54),
 ((('подать', 'obj'), ('подать', 'obj'), 'голова'), 52),
 ((('делать', 'obj'), ('делать', 'obj'), 'конь'), 50),
 ((('говорить', 'nsubj'), ('просить', 'obj'), 'образ'), 45),
 ((('достать', 'obj'), ('достать', 'obj'), 'конь'), 42),
 ((('бежать', 'nsubj'), ('бежать', 'nsubj'), 'конь'), 39),
 ((('делать', 'obj'), ('бежать', 'nsubj'), 'конь'), 37),
 ((('говорить', 'nsubj'), ('дать', 'obj'), 'образ'), 36),
 ((('говорить', 'nsubj'), ('видеть', 'obj'), 'образ'), 36)]

In [ ]:
for i in [ft.gen2, ft.gen1]:
  if i is ft.gen2 or i is ft.gen1:
    print(i[1])

0
1


In [ ]:
wtf = {'gen1': ['-', '-'], 'gen2': ['-', '-'],
       'num1': ['-', '-'], 'num2': ['-', '-'],
       'pers1': ['-', '-'], 'pers2': ['-', '-'],
       'anim1': ['-', '-'], 'anim2': ['-', '-']}

In [ ]:
tf = pd.DataFrame.from_dict(wtf)
tf

,gen1,gen2,num1,num2,pers1,pers2,anim1,anim2
0,-,-,-,-,-,-,-,-
1,-,-,-,-,-,-,-,-


In [ ]:
table_fix(tf)

In [ ]:
tf

,gen1,gen2,num1,num2,pers1,pers2,anim1,anim2
0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0


In [ ]:
for c in [('c', tf.c1, 'c1'), ('c', tf.c2, 'c2')]:
  if c[2].startswith('c'):
    fs = []
    for item in c[1]:
      fs.append(6)
    print(fs)
    if c[2] == 'c1':
      tf.c1 = fs
    elif c[2] == 'c2':
      tf.c2 = fs

[6, 6]
[6, 6]


In [ ]:
import os

In [ ]:
coref_event_count = []
coref_role_count = []
all_events = []
event_counter = 0
for t in f_texts:
  ft, ev_chain, ft_table = text_preproc(t)
  try:
    coref_chains, text_events = get_coref_chains(ft, ev_chain, ft_table)
  except LookupError:
    print('sorry')
    continue
  event_counter += len(ev_chain)
  all_events.extend(text_events)
  add_data(coref_chains, coref_event_count, coref_role_count)
  print('ready')

ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
sorry
ready
ready
ready


In [ ]:
f_texts = []
dir_name = '293-307'
for filename in os.listdir(dir_name):
  #print(filename)
  with open(os.path.join(dir_name, filename), 'r', encoding='utf-8') as f:
    text = f.read()
    f_texts.append(text)

In [ ]:
f_texts = []
with open(os.path.join("fairytales2", '105.txt'), 'r', encoding='utf-8') as f:
  text = f.read()
  f_texts.append(text)

In [ ]:
len(f_texts)

14

In [ ]:
for t in f_texts:
  ft, ev_chain, ft_table = text_preproc(t)
  try:
    coref_chains, text_events = get_coref_chains(ft, ev_chain, ft_table)
  except LookupError:
    print('sorry')
    continue
  event_counter += len(ev_chain)
  all_events.extend(text_events)
  add_data(coref_chains, coref_event_count, coref_role_count)
  print('ready')

ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready
ready


In [ ]:
event_counter

2675

In [ ]:
all_events[-1]

('наживать', 'obj')

In [ ]:
with open('coref_event_count.txt', 'a', encoding='utf-8') as f:
  for pair in coref_event_count:
    f.write(pair[0][0]+'\t'+pair[0][1]+'\t'+pair[1][0]+'\t'+pair[1][1]+'\n')

In [ ]:
with open('coref_role_count.txt', 'a', encoding='utf-8') as f:
  for pair in coref_role_count:
    f.write(pair[0][0]+'\t'+pair[0][1]+'\t'+pair[1][0]+'\t'+pair[1][1]+'\t'+pair[2]+'\n')

In [ ]:
with open('all_events.txt', 'a', encoding='utf-8') as f:
  for pair in all_events:
    f.write(pair[0]+'\t'+pair[1]+'\n')

In [ ]:
with open('event_count.txt', 'a', encoding='utf-8') as f:
  f.write('\n'+str(event_counter))

СХЕМЫ????

In [ ]:
pmi(pair, c_coref_event_count, c_all_events, event_counter, len_coref_event)

In [ ]:
def sim(pair, argument, lmbd=1):
  pair_pmi = pmi(pair, c_coref_event_count, c_all_events, event_counter, len_coref_event)
  if c_coref_role_count[(pair[0], pair[1], argument)] > 0:
    result = pair_pmi + (lmbd * math.log(c_coref_role_count[(pair[0], pair[1], argument)], 2))
  else:
    result = pair_pmi
  return result

In [ ]:
[11, 3][1:]

[3]

In [ ]:
def chain_score(chain, argument, lmbd=1):
  if len(chain) > 1:
    score = 0
    for event1 in chain[:-1]:
      for event2 in chain[1:]:
        score += sim((event1, event2), argument, lmbd=lmbd)
  else:
    freq = 0
    for key in c_coref_role_count:
      if argument in key and chain[0] in key:
        freq += c_coref_role_count[key]
    if freq != 0:
      score = lmbd * math.log(freq, 2)
    else:
      score = 0
  
  return score

In [ ]:
c_coref_role_count = Counter(coref_role_count)

In [ ]:
c_coref_role_count.most_common(5)

[((('говорить', 'nsubj'), ('говорить', 'nsubj'), 'Никита'), 72),
 ((('принести', 'obj'), ('принести', 'obj'), 'войско'), 72),
 ((('говорить', 'nsubj'), ('говорить', 'nsubj'), 'Иван'), 64),
 ((('сесть', 'nsubj'), ('сесть', 'nsubj'), 'Иван'), 64),
 ((('влезть', 'nsubj'), ('влезть', 'nsubj'), 'дурак'), 61)]

In [ ]:
#collecting all arguments
arg_list = []
for key in c_coref_role_count:
  if key[-1] not in arg_list:
    arg_list.append(key[-1])

In [ ]:
all_events[:5]

[('делать', 'nsubj'),
 ('делать', 'nsubj'),
 ('гладеть', 'obj'),
 ('угождать', 'nsubj'),
 ('сказать', 'obj')]

In [ ]:
#chainsim for 1 event
def event_arg_similarity(chain, new_ev, arg_list, lmbd=1):
  best_arg = None
  sim_sc = 0
  for argument in arg_list:
    arg_sc = chain_score(chain, argument, lmbd=lmbd)
    new_ev_sc = 0
    for ev in chain:
      new_ev_sc += sim((ev, new_ev), argument, lmbd=lmbd)
    if (arg_sc + new_ev_sc) > sim_sc:
      best_arg = argument
      sim_sc = arg_sc + new_ev_sc
  return sim_sc, best_arg

In [ ]:
the_best_sc = 0
the_best_arg = None
the_best_event = None
for event in tqdm(c_all_events):
  sim_sc, best_arg = event_arg_similarity([('купаться', 'nsubj'),], event, arg_list, lmbd=1)
  if sim_sc > the_best_sc:
    the_best_sc = sim_sc
    the_best_arg = best_arg
    the_best_event = event

 19%|█▉        | 1003/5192 [8:59:42<34:03:28, 29.27s/it]

In [ ]:
# collecting all verbs
verb_list = []
for event in all_events:
  if event[0] not in verb_list:
    verb_list.append(event[0])

In [ ]:
verb_list[:5]

['делать', 'гладеть', 'угождать', 'сказать', 'купаться']

In [ ]:
dep_types = ['nsubj', 'obj']
c_coref_event_count = Counter(coref_event_count)
c_all_events = Counter(all_events)
len_coref_event = len(coref_event_count)

In [ ]:
nar_scheme = [[[[('купаться', 'nsubj'),], []],], ['купаться',]]
nar_scheme[0]

[[[('купаться', 'nsubj')], []]]

In [ ]:
nar_scheme = [[[[('купаться', 'nsubj'),], []],], 
              ['купаться',]]

new_args = []
new_verb = None
max_score = 0
for verb in verb_list:
  if verb not in nar_scheme[-1]:
    verb_score = 0
    for dep in dep_types:
      event_sc = 0
      for chain in nar_scheme[0]:
        sim_sc, best_arg = event_arg_similarity(chain[0], (verb, dep), arg_list, lmbd=1)
        if sim_sc >= event_sc:
          event_sc = sim_sc
      verb_score += event_sc
    if verb_score > max_score:
      new_verb = verb
      max_score = verb_score

In [ ]:
event_arg_similarity([('купаться', 'nsubj'),], ('делать', 'nsubj'), arg_list, lmbd=1)

(0, None)